# [Matplotlib](https://matplotlib.org/) Tutorial
stough, 202-

[Matplotlib](https://matplotlib.org/) is a highly customizable visualization toolkit for Python. So highly customizable in terms of fonts, interactivity, and animations that we cannot possibly cover it in a short notebook like this. For more complete coverage and documentation you should definitely look to the [matplotlib tutorials](https://matplotlib.org/tutorials/index.html) online. 

As we use matplotlib constantly and in different ways throughout image processing, most of these usage cases shall be left to the moment we need them. This notebook is meant only to serve as an introduction with more explication.

1. [Loading and Displaying Images](#load_display)
1. [Plotting Image-Derived Information](#plot_info)
  1. [Histograms](#histograms)
  1. [3D plotting](#3d_plotting)

## Imports

- The [`%matplotlib widget` magic command](https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-matplotlib) starts up the matplotlib visualization package for interactive plotting capability through [ipympl](https://github.com/matplotlib/ipympl). We'll be using `widget` usually. Alternatives include `inline` for static figures (can be very useful to debug) and `notebook` if we were using classic Jupyter Notebook.
- The business with `sys.path.insert` is to allow us to import modules from our utility directory (read more [here](https://stackoverflow.com/questions/49264194/import-py-file-in-another-directory-in-jupyter-notebook)).

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np

# For importing from alternative directory sources
import sys  
sys.path.insert(0, '../dip_utils')

import matrix_utils

<a id='load_display'></a>
## Loading and Displaying Images
We can use matplotlib's [`pyplot.imread`](https://matplotlib.org/3.3.3/api/_as_gen/matplotlib.pyplot.imread.html) to load images from our file system or even a URL. If successful, the function returns a numpy array (or really `ndarray`). To test here, we'll read in one of our book photos.

In [ ]:
I = plt.imread('../dip_pics/cat_small.png')

In [ ]:
type(I)

In [ ]:
matrix_utils.arr_info(I)

Using our `arr_info` utility, we can see that the result is a 512x512x4 matrix of floating point numbers in the range [0,1]. The 512x512 represents the image space, while the x4 means there are four *channels* in this image. Normally, these four channels represent red, green, blue, and *alpha*, or opacity. In this case alpha is all 1, which means no transparency. 

To view the image we'll use the [`pyplot.imshow`](https://matplotlib.org/3.3.3/api/_as_gen/matplotlib.pyplot.imshow.html) function. If we're making multiple plots in a single notebook (usually the case), we'll also want to instantiate a figure in which to do the plotting, using [`pyplot.figure`](https://matplotlib.org/3.3.3/api/_as_gen/matplotlib.pyplot.figure.html).

In [ ]:
plt.figure(figsize=(4,4))
plt.imshow(I) # add semicolon ; to silense the printed output.

Play with the interactively plotted image above. You should be able to resize it (lower-right corner) or zoom into a particular area (little square button), or a number of other interactions all through click-and-drag.

**Important**

Copying from the [`pyplot.imshow`](https://matplotlib.org/3.3.3/api/_as_gen/matplotlib.pyplot.imshow.html) documentation, the image data array *must* be one of the following:
```
- (M, N): an image with scalar data. The data is visualized using a colormap.
- (M, N, 3): an image with RGB values (0-1 float or 0-255 int).
- (M, N, 4): an image with RGBA values (0-1 float or 0-255 int), i.e. including transparency.
```

To be specific, floating point pixel values outside of [0,1] may lead to an error, and integer pixel values outside the [0,255] range may be clipped or else even cast (e.g., a pixel value of 256 might be interpretted as 0, wrapping around the [0,255] range), leading to screwy output. So use that `arr_info` function often.


The image matrix `I` has three color channels. Each pixel color is actually represented by three color stimuli in red, green, and blue. While we discuss this in more detail elsewhere, for now you can think of `I` as really a red image, green image, and blue image all pasted together. For the purposes of showing off matplotlib, we'll display these three separate images.

In [ ]:
f, ax = plt.subplots(1,3,figsize=(7,2))
ax[0].imshow(I[...,0], cmap='Reds') # White to deep red.
ax[1].imshow(I[...,1], cmap='Greens')
ax[2].imshow(I[...,2], cmap='Blues')

As you can see, we used [`subplots`](https://matplotlib.org/3.3.3/api/_as_gen/matplotlib.pyplot.subplots.html) to create a figure with a one-by-three grid of axes, which is then returned as a list. We then call each axis' [`imshow`](https://matplotlib.org/3.3.3/api/_as_gen/matplotlib.axes.Axes.imshow.html) method to plot a different channel of the image. Some notes:
- `figsize` is in width by height as a tuple, and would be in inches only if all the other non-Python elements of the display pipeline are calibrated appropriately. This is a parameter you'll likely have to play with.
- There are scores of [colormaps](https://matplotlib.org/tutorials/colors/colormaps.html) available to you through the `cmap` parameter. Choosing which colormap might be appropriate is a much larger discussion that we leave for later.
- The coordinate system shown for each axis has the origin **(0,0) at the upper left corner**. This is the matrix or image coordinate system, in contrast to the spatial system you have perhaps internalized, with the origin in the lower left. The difference between the `ij` and `xy` systems can be important when you plot multiple data sources (images and non-image data, like points) on the same axis.   
- Every time you execute the above cell, you generate a *new* figure. Even though the old ones become inaccessible, the active python kernel is keeping track of them. Execute the cell too many times and you will receive a warning about potential memory usage. This is not usually a problem, but be sure not to put `plt.figure()` into large loop unless you're sure that's what you want.

In [ ]:
f, ax = plt.subplots(1,4,figsize=(8,2), sharex=True, sharey=True)
ax[0].imshow(I)
ax[0].set_title('image')
ax[1].imshow(I[...,0], cmap='Reds_r') # black to red colormap
ax[1].set_title('red channel')
ax[2].imshow(I[...,1], cmap='Greens_r')
ax[2].set_title('green')
ax[3].imshow(I[...,2], cmap='Blues_r')
ax[3].set_title('blue')

[a.axes.get_xaxis().set_visible(False) for a in ax];
[a.axes.get_yaxis().set_visible(False) for a in ax];

plt.tight_layout()

In the above cell I add several additional pieces for better visualization. 
- Setting `sharex` and `sharey` allow us to zoom and interact with all subplots identically. Give it a shot, it can be quite useful for viewing regions that in the original display might be too small to observe. For example, if you zoom into an area of orange, you might note the constituent red > green > blue. 
- I've changed the colormaps used from the ones in the previous figure. I thought a colormap that would make the cat's eyes dark would appear a little more natural. Again, read more about [colormaps](https://matplotlib.org/tutorials/colors/colormaps.html).
- Image values are shown at the bottom of the figure as you mouse over. Note that the original image, shown in the left-most axis, has all four channels listed.
- I've added titles to each axis using the [`set_title`](https://matplotlib.org/api/_as_gen/matplotlib.axes.Axes.set_title.html) method. Highly recommended to communicate to the reader.
- I removed the axis markings, which allows more space for the image content I'm trying to display. Try commenting out the lines that begin `[a.axes.get_...` and execute the cell again to see the difference.
- [`tight_layout`](https://matplotlib.org/api/tight_layout_api.html) reduces padding between axes, giving even more space for content. Try commenting out the line and re-executing to see the difference. 

<a id='plot_info'></a>
## Plotting Image-derived Information

Maplotlib isn't only about plotting image data of course. Sometimes you want to plot data *about the images*! While there are any number of ways to look at imaging data, two that we will regularly use are histograms and a color box. While both methods throw away important spatial relationships between pixels, they still turn out to be useful, especially as we consider color and contrast enhancements. 

Let's take a look at histograms first. For fun we'll use a different image.

In [ ]:
J = plt.imread('../dip_pics/sf.jpg')
matrix_utils.arr_info(J)

`arr_info` tells us this image is different from the previous. First, it only has three channels, again for the red/green/blue colors but missing the alpha/transparency channel that `I` had. Second, the pixels are not floating point values anymore, but rather 8-bit unsigned integer. Lastly, we can also see that the pixel values cover the entire possible 8-bit range, [0,255]. Well, let's take a look...

In [ ]:
plt.figure(figsize=(4,3))
plt.imshow(J); # semicolon ; to suppress text display

As you mouse over the image above you can see the x and y location of the mouse in the image space, along with the `[red, green, blue]` values of the pixel at that location.

<a id='histograms'></a>
### Histograms

Pyplot has its own histogramming function [`pyplot.hist`](https://matplotlib.org/3.3.3/api/_as_gen/matplotlib.pyplot.hist.html) that can return histogram data (bin counts, bins, and underlying visual objects if needed) and display that histogram at the same time. [`numpy.histogram`](https://numpy.org/doc/stable/reference/generated/numpy.histogram.html) can also be used if needed.

In [ ]:
plt.figure(figsize=(5,3))
n, allbins, _ = plt.hist(J.ravel(), bins=np.arange(256))
plt.title("Overall Histogram");

In the above I call [`ravel`](https://numpy.org/doc/stable/reference/generated/numpy.ravel.html) to give a 1D representation of the image data to `hist`. By the way, [`flatten`](`https://numpy.org/doc/stable/reference/generated/numpy.ndarray.flatten.html) would do the same thing while also making a copy of the data, unneeded here. 

I also told `hist` what the bins should be through the `bins` argument. `np.arange(256)` returns an array of the sequence [0,...,255]. We can also display each of the color channels separately, which can be a very informative way of understanding the relative strength of the different colors in the image...

In [ ]:
f, axarr = plt.subplots(1,2, figsize=(9, 3))

axarr[0].imshow(J) #https://matplotlib.org/api/_as_gen/matplotlib.pyplot.imshow.html
axarr[0].set_title('Image')

axarr[1].hist(J[...,0].ravel(), allbins, alpha = .6, label = 'red', color = 'r');
axarr[1].hist(J[...,1].ravel(), allbins, alpha = .6, label = 'green', color = 'g');
axarr[1].hist(J[...,2].ravel(), allbins, alpha = .6, label = 'blue', color = 'b');
axarr[1].legend(loc = 'upper right');
plt.tight_layout()

In the above cell we overlay histograms for each color channel on the same axis. We use the `alpha` transparency parameter to see all three at the same time, `label` to populate a legend, and `color` to set the obvious color (see [`pyplot.hist`](https://matplotlib.org/3.3.3/api/_as_gen/matplotlib.pyplot.hist.html) for all the details). [`legend`](https://matplotlib.org/3.3.3/api/_as_gen/matplotlib.pyplot.legend.html) can then automatically generate a legend with our labels. 

<a id='3d_plotting'></a>
### 3D Plotting

In addition to ignoring the spatial relationships between pixels, `hist` also separates the red, green, and blue components from on another. Another interesting visualization that keeps all of a pixel's color together is the color cube. I'm showing this here only because it's an interesting use case for 3D plotting. 

In [ ]:
NUMPOINTS = 5000

fig = plt.figure(figsize=(5,5))
ax = fig.add_subplot(111, projection='3d')

# X is the N*M x 3 version of the image.
X = np.stack([J[...,i].ravel() for i in range(3)]).T

# https://numpy.org/doc/stable/reference/random/generated/numpy.random.choice.html
randomInds = np.random.choice(np.arange(X.shape[0]), NUMPOINTS, replace=False)

# Now plot those pixels in the 3d space.
ax.scatter(X[randomInds,0], X[randomInds,1], X[randomInds,2], c=X[randomInds, :]/255)

# Label the axes.
ax.set_xlabel('Red')
ax.set_ylabel('Green')
ax.set_zlabel('Blue');

You can click-and-drag the above visualization to change the view of the data. This has been a brief introduction to plotting in matplotlib for the purposes of image processing preparedness. You can read more about 3D plotting [here](https://jakevdp.github.io/PythonDataScienceHandbook/04.12-three-dimensional-plotting.html) or [here](https://numpy.org/doc/stable/reference/random/generated/numpy.random.choice.html), or other plotting through the [matplotlib tutorials](https://matplotlib.org/tutorials/index.html). Have fun!